In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from lfb import fetch_season_games_teams, fetch_date_infos, add_fiba_id, fetch_players, data_path
from data import save_data, load_data
%load_ext autoreload
%autoreload 2

# Méthodologie

Trouver les équipes : scraper les pages de matchs et ne garder que les nouveaux id.  
Trouver les joueuses : scraper les pages de match et faire un list set.  
Trouver les matchs : boucle sur la date / liste des matchs / récupérer les liens.  
Accéder au livestat : lien du game + 'cliquer' sur livestats (pas les 5D dans la page LFB) en récupérant l'id du lien.  
A partir du lien : les 5D et le PBP se trouvent. 
 
 

## Récupération des matchs

In [11]:
load = False
season = 2021
competition = 'lfb'

In [12]:
if load :
    games = fetch_season_games_teams(season)
    games = add_fiba_id(games)
else :
    games_path = data_path+'games_'+competition+'_'+str(season)+'.csv'
    games = pd.read_csv(games_path,index_col=False).drop(columns = ['Unnamed: 0'])

## Obtention de la liste des plays et qualifiers de la saison - ainsi que les erreurs (matchs sans json)

In [13]:
type(games.iloc[14]['fiba_game_id'])

float

In [14]:
plays, qualifiers, errors = find_possible_plays(games)

In [15]:
plays_list = ['3pt_made','3pt_missed','2pt_made','2pt_missed','freethrow_made','freethrow_missed','off_rebound','def_rebound','assist','block','steal', 'turnover','foul','foulon']
qualifiers_list = ['shooting','pointsinthepaint','fromturnover','fastbreak']

In [16]:
games.iloc[errors]

,date,home,away,lfb_game_id,fiba_game_id
7,2021-10-09,Angers,Tarbes,/match/666476-angers-tarbes,https://livestats.dcd.shared.geniussports.com/...
14,2021-10-16,Saint-Amand,Landerneau,/match/666483-saint-amand-landerneau,NaN
67,2022-01-16,Landerneau,Lyon,/match/666539-landerneau-lyon,https://livestats.dcd.shared.geniussports.com/...
71,2022-01-30,Saint-Amand,Roche Vendée,/match/666543-saint-amand-roche-vendee,https://livestats.dcd.shared.geniussports.com/...
89,2022-03-05,Angers,Lyon,/match/666563-angers-lyon,https://livestats.dcd.shared.geniussports.com/...


## Récupérer et traiter le play by play

In [17]:
# Fetching JSON file.

load = False

if load :
    url_match_fiba = games.iloc[104]['fiba_game_id']
    reponse = requests.get(url_match_fiba, timeout = 10)
    game_detail = reponse.json()
    pbp = pd.DataFrame(game_detail['pbp']).sort_values('actionNumber', ascending = True)
    pbp.set_index('actionNumber', inplace=True)
    pbp.drop(columns = ['player','shirtNumber','internationalFirstName', 'internationalFamilyName', 'firstNameInitial',
        'familyNameInitial', 'internationalFirstNameInitial',
        'internationalFamilyNameInitial', 'scoreboardName'],inplace = True)
    pbp['player']=pbp['firstName']+ ' ' + pbp['familyName']
    pbp.drop(columns=['firstName','familyName'], inplace = True)
else :   
    pbp = pd.read_csv(data_path+'mock_pbp.csv').set_index('actionNumber')
pbp

,gt,clock,s1,s2,lead,tno,period,periodType,pno,success,actionType,previousAction,qualifier,subType,scoring,player
actionNumber,,,,,,,,,,,,,,,,
1,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,game,NaN,[],start,0,NaN
2,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,period,NaN,[],start,0,NaN
4,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,jumpball,NaN,[],startperiod,0,NaN
5,09:58,09:58:00,0,0,0,2,1,REGULAR,9,1,jumpball,4.0,[],won,0,Kayla Alexander
6,09:58,09:58:00,0,0,0,1,1,REGULAR,1,1,jumpball,4.0,[],lost,0,Ana Tadic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,00:07,00:07:40,66,63,3,1,4,REGULAR,2,0,freethrow,NaN,[],2of2,1,Clementine Samson
631,00:07,00:07:40,66,63,3,2,4,REGULAR,6,1,rebound,629.0,[],defensive,0,Alexia Chartereau
633,00:02,00:02:80,66,63,3,2,4,REGULAR,6,0,3pt,NaN,[],jumpshot,1,Alexia Chartereau


In [18]:
shoots = ['2pt','3pt','freethrow']
direct_translation = ['foul','foulon', 'assist','steal','turnover', 'block']

def map_play(row):
    res = ''
    if row['actionType'] not in plays : 
        res = 'ignored'
    elif row['actionType'] in direct_translation:
        res = row['actionType']
    elif row['actionType'] in shoots:
        if row['success'] == 1 :
            res = row['actionType']+'_made'
        else :
            res = row['actionType']+'_missed'
    elif row['actionType'] == 'rebound':
        if row['subType'] == 'defensive':
            res = 'def_rebound'
        else :
            res = 'off_rebound'
    
    return res

In [19]:
test =  []

for index,row in pbp.iterrows():
    test.append(map_play(row))

pbp['mapped_play'] = test

In [20]:
pbp.query("actionType == 'period' ")

,gt,clock,s1,s2,lead,tno,period,periodType,pno,success,actionType,previousAction,qualifier,subType,scoring,player,mapped_play
actionNumber,,,,,,,,,,,,,,,,,
2,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,period,NaN,[],start,0,NaN,ignored
152,00:00,00:00:00,20,23,-3,0,1,REGULAR,0,1,period,NaN,['confirmed'],end,0,NaN,ignored
155,10:00,10:00:00,20,23,-3,0,2,REGULAR,0,1,period,NaN,[],start,0,NaN,ignored
316,00:00,00:00:00,37,32,5,0,2,REGULAR,0,1,period,NaN,['confirmed'],end,0,NaN,ignored
323,10:00,10:00:00,37,32,5,0,3,REGULAR,0,1,period,NaN,[],start,0,NaN,ignored
472,00:00,00:00:00,53,50,3,0,3,REGULAR,0,1,period,NaN,['confirmed'],end,0,NaN,ignored
475,10:00,10:00:00,53,50,3,0,4,REGULAR,0,1,period,NaN,[],start,0,NaN,ignored
636,00:00,00:00:00,66,63,3,0,4,REGULAR,0,1,period,NaN,['confirmed'],end,0,NaN,ignored


In [21]:
pbp['poss_time_index'] = pbp['periodType']+'_'+[str(item) for item in pbp['period']]+'_'+pbp['clock']
pbp

,gt,clock,s1,s2,lead,tno,period,periodType,pno,success,actionType,previousAction,qualifier,subType,scoring,player,mapped_play,poss_time_index
actionNumber,,,,,,,,,,,,,,,,,,
1,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,game,NaN,[],start,0,NaN,ignored,REGULAR_1_10:00:00
2,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,period,NaN,[],start,0,NaN,ignored,REGULAR_1_10:00:00
4,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,jumpball,NaN,[],startperiod,0,NaN,,REGULAR_1_10:00:00
5,09:58,09:58:00,0,0,0,2,1,REGULAR,9,1,jumpball,4.0,[],won,0,Kayla Alexander,,REGULAR_1_09:58:00
6,09:58,09:58:00,0,0,0,1,1,REGULAR,1,1,jumpball,4.0,[],lost,0,Ana Tadic,,REGULAR_1_09:58:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,00:07,00:07:40,66,63,3,1,4,REGULAR,2,0,freethrow,NaN,[],2of2,1,Clementine Samson,freethrow_missed,REGULAR_4_00:07:40
631,00:07,00:07:40,66,63,3,2,4,REGULAR,6,1,rebound,629.0,[],defensive,0,Alexia Chartereau,def_rebound,REGULAR_4_00:07:40
633,00:02,00:02:80,66,63,3,2,4,REGULAR,6,0,3pt,NaN,[],jumpshot,1,Alexia Chartereau,3pt_missed,REGULAR_4_00:02:80


In [22]:
equipe = 'Lyon'
team_index = 2

url_match_fiba = games.iloc[104]['fiba_game_id']
game_test = fetch_game_json(url_match_fiba)
s5,pl = compile_starting_5(game_test,team_index)

In [23]:
plays_list

['3pt_made',
 '3pt_missed',
 '2pt_made',
 '2pt_missed',
 'freethrow_made',
 'freethrow_missed',
 'off_rebound',
 'def_rebound',
 'assist',
 'block',
 'steal',
 'turnover',
 'foul',
 'foulon']

## <span color = 'red'> Mapping poss a revoir </span>

In [24]:
pl_dict = {player : 0 for player in pl}
for player in s5:
    pl_dict[player]=1
    
on_floor_dict = {}
offense = {}

gain_poss_plays = ['def_rebound','off_rebound','steal']
loose_poss_plays = ['3pt_made','2pt_made','turnover']
poss_off = 0
init_str = 'REGULAR_1_10:00:00'
loose_poss = False
gain_poss = False


for index, row in pbp.iterrows():
    
    # gestion des joueur.se.s sur le terrain
    on_floor_dict[index] = pl_dict.copy()
    if pbp.loc[index, 'actionType'] == 'substitution' and pbp.loc[index,'tno'] == team_index:
        if row['subType'] == 'in':
            pl_dict[row['player']] = 1 
        else :
            pl_dict[row['player']] = 0 
            
    # gestion des possession              
    end_str = row['poss_time_index']    
    
    if not end_str == init_str:
        poss_off = max(min(poss_off + gain_poss - loose_poss,1),0)
        #print(f'{index}, {init_str} - {end_str}, loose = {loose_poss}, gain = {gain_poss}, poss = {poss_off}')
        loose_poss = False
        gain_poss = False 
    
    offense[index] = poss_off
    init_str = end_str    
    if row['actionType'] == 'jumpball' and row['subType'] == 'won' and row['tno'] == team_index :
        gain_poss = True
    elif row['mapped_play'] == 'freethrow_made' and (row['subType'] == '1of1' or row['subType'] == '2of2' or row['subType'] == '3of3'):
        loose_poss = True   
    elif row['mapped_play'] in gain_poss_plays and row['tno'] == team_index:
        gain_poss = True
    elif row['mapped_play'] in loose_poss_plays and row['tno'] != team_index:
        gain_poss = True
    elif row['mapped_play'] in gain_poss_plays and row['tno'] != team_index:
        loose_poss = True 
    elif row['mapped_play'] in loose_poss_plays and row['tno'] == team_index:
        loose_poss = True     
    
    #print(f'{index} - loose = {loose_poss} / gain = {gain_poss}')
    
         

In [25]:
changes = []
st_key = 1

for key,value in offense.items():
    if key == st_key:
        pass
    if not offense[st_key] == offense[key]:
        changes.append(key)
    st_key = key

print(f'Nombre de changements de possessions : {len(changes)}')
print(changes)

Nombre de changements de possessions : 137
[7, 10, 13, 21, 23, 25, 33, 36, 41, 43, 45, 50, 57, 61, 63, 65, 66, 67, 72, 81, 92, 103, 111, 113, 116, 118, 119, 121, 123, 125, 128, 143, 145, 149, 152, 159, 168, 174, 187, 194, 197, 210, 213, 218, 223, 225, 227, 230, 237, 240, 273, 276, 278, 280, 282, 284, 285, 299, 309, 311, 316, 331, 335, 340, 348, 357, 358, 366, 368, 370, 372, 373, 374, 377, 380, 382, 385, 389, 391, 392, 394, 399, 401, 403, 405, 407, 409, 412, 422, 424, 426, 429, 438, 441, 443, 445, 447, 452, 468, 479, 484, 486, 488, 490, 492, 494, 496, 506, 508, 512, 515, 517, 521, 523, 525, 534, 543, 545, 548, 555, 556, 558, 561, 570, 575, 581, 590, 592, 594, 597, 600, 602, 605, 611, 615, 618, 633]


In [26]:
switch = 600
st = switch - 5
end = switch + 5

display(pbp.loc[st:end])
for i in range(st,end+1):
    if offense.get(i) is not None:
        print(f'{i} - {offense.get(i)}') 

,gt,clock,s1,s2,lead,tno,period,periodType,pno,success,actionType,previousAction,qualifier,subType,scoring,player,mapped_play,poss_time_index
actionNumber,,,,,,,,,,,,,,,,,,
595,02:17,02:17:00,63,60,3,1,4,REGULAR,3,1,rebound,594.0,[],defensive,0,Marie Pardon,def_rebound,REGULAR_4_02:17:00
597,01:55,01:55:00,63,60,3,1,4,REGULAR,0,1,turnover,NaN,['team'],24sec,0,NaN,turnover,REGULAR_4_01:55:00
600,01:44,01:44:00,63,60,3,2,4,REGULAR,12,1,turnover,NaN,[],badpass,0,Julie Allemand,turnover,REGULAR_4_01:44:00
602,01:24,01:24:00,63,60,3,1,4,REGULAR,2,0,3pt,NaN,['fromturnover'],jumpshot,1,Clementine Samson,3pt_missed,REGULAR_4_01:24:00
603,01:19,01:19:00,63,60,3,2,4,REGULAR,12,1,rebound,602.0,[],defensive,0,Julie Allemand,def_rebound,REGULAR_4_01:19:00
605,01:15,01:15:00,63,62,1,2,4,REGULAR,19,1,2pt,NaN,"['pointsinthepaint', 'fastbreak']",layup,1,Cassidie Burdick,2pt_made,REGULAR_4_01:15:00


595 - 1
597 - 0
600 - 1
602 - 0
603 - 0
605 - 1


In [27]:
on_floor_dict[129]

{'Juste Jocyte': 0,
 'Ingrid Tanqueray': 1,
 'Aleksandra Crvendakic': 1,
 'Alexia Chartereau': 1,
 'Dominique Malonga': 0,
 'Julie Allemand': 0,
 'Marine Johannes': 0,
 'Marieme Badiane': 1,
 'Cassidie Burdick': 0,
 'Kayla Alexander': 0,
 'Sara Chevaugeon': 1}

In [28]:
pbp.query("actionType == 'jumpball'")

,gt,clock,s1,s2,lead,tno,period,periodType,pno,success,actionType,previousAction,qualifier,subType,scoring,player,mapped_play,poss_time_index
actionNumber,,,,,,,,,,,,,,,,,,
4,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,jumpball,NaN,[],startperiod,0,NaN,,REGULAR_1_10:00:00
5,09:58,09:58:00,0,0,0,2,1,REGULAR,9,1,jumpball,4.0,[],won,0,Kayla Alexander,,REGULAR_1_09:58:00
6,09:58,09:58:00,0,0,0,1,1,REGULAR,1,1,jumpball,4.0,[],lost,0,Ana Tadic,,REGULAR_1_09:58:00
10,09:26,09:26:00,0,2,-2,1,1,REGULAR,0,1,jumpball,NaN,[],heldball,0,NaN,,REGULAR_1_09:26:00
179,08:10,08:10:00,22,28,-6,1,2,REGULAR,0,1,jumpball,NaN,[],heldball,0,NaN,,REGULAR_2_08:10:00


In [29]:
pbp.loc[:15]

,gt,clock,s1,s2,lead,tno,period,periodType,pno,success,actionType,previousAction,qualifier,subType,scoring,player,mapped_play,poss_time_index
actionNumber,,,,,,,,,,,,,,,,,,
1,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,game,NaN,[],start,0,NaN,ignored,REGULAR_1_10:00:00
2,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,period,NaN,[],start,0,NaN,ignored,REGULAR_1_10:00:00
4,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,jumpball,NaN,[],startperiod,0,NaN,,REGULAR_1_10:00:00
5,09:58,09:58:00,0,0,0,2,1,REGULAR,9,1,jumpball,4.0,[],won,0,Kayla Alexander,,REGULAR_1_09:58:00
6,09:58,09:58:00,0,0,0,1,1,REGULAR,1,1,jumpball,4.0,[],lost,0,Ana Tadic,,REGULAR_1_09:58:00
7,09:45,09:45:00,0,2,-2,2,1,REGULAR,9,1,2pt,NaN,['pointsinthepaint'],layup,1,Kayla Alexander,2pt_made,REGULAR_1_09:45:00
8,09:45,09:45:00,0,2,-2,2,1,REGULAR,8,1,assist,7.0,[],NaN,0,Aleksandra Crvendakic,assist,REGULAR_1_09:45:00
10,09:26,09:26:00,0,2,-2,1,1,REGULAR,0,1,jumpball,NaN,[],heldball,0,NaN,,REGULAR_1_09:26:00
12,09:22,09:22:00,2,2,0,1,1,REGULAR,12,1,2pt,NaN,['pointsinthepaint'],drivinglayup,1,Julie Wojta,2pt_made,REGULAR_1_09:22:00


In [30]:
bin_plays = pd.DataFrame(index = pbp.index, columns = plays_list+qualifiers_list)
bin_plays.fillna(0, inplace = True)

for column in plays_list:
    bin_plays[column] = [1 if item == column else 0 for item in pbp['mapped_play']]
    
for column in qualifiers_list:
    bin_plays[column] = [1 if column in item else 0 for item in pbp['qualifier']]

bin_plays

,3pt_made,3pt_missed,2pt_made,2pt_missed,freethrow_made,freethrow_missed,off_rebound,def_rebound,assist,block,steal,turnover,foul,foulon,shooting,pointsinthepaint,fromturnover,fastbreak
actionNumber,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
631,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
633,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
dict_grouping = {item : 'sum' for item in plays_list+qualifiers_list}
tmp_play = pbp.merge(bin_plays, left_index = True, right_index=True)
tmp_play = tmp_play.groupby('poss_time_index').agg(dict_grouping)
tmp_play['sum_act']= tmp_play.sum(axis=1)
no_action = tmp_play.query("sum_act == 0").index

In [32]:
no_action

Index(['REGULAR_1_00:00:00', 'REGULAR_1_03:26:00', 'REGULAR_1_09:26:00',
       'REGULAR_1_09:58:00', 'REGULAR_1_10:00:00', 'REGULAR_2_00:00:00',
       'REGULAR_2_03:36:00', 'REGULAR_2_07:51:00', 'REGULAR_2_10:00:00',
       'REGULAR_3_00:00:00', 'REGULAR_3_10:00:00', 'REGULAR_4_00:00:00',
       'REGULAR_4_08:17:00', 'REGULAR_4_10:00:00'],
      dtype='object', name='poss_time_index')

In [33]:
pbp[[item in no_action for item in pbp['poss_time_index']]]

,gt,clock,s1,s2,lead,tno,period,periodType,pno,success,actionType,previousAction,qualifier,subType,scoring,player,mapped_play,poss_time_index
actionNumber,,,,,,,,,,,,,,,,,,
1,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,game,NaN,[],start,0,NaN,ignored,REGULAR_1_10:00:00
2,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,period,NaN,[],start,0,NaN,ignored,REGULAR_1_10:00:00
4,10:00,10:00:00,0,0,0,0,1,REGULAR,0,1,jumpball,NaN,[],startperiod,0,NaN,,REGULAR_1_10:00:00
5,09:58,09:58:00,0,0,0,2,1,REGULAR,9,1,jumpball,4.0,[],won,0,Kayla Alexander,,REGULAR_1_09:58:00
6,09:58,09:58:00,0,0,0,1,1,REGULAR,1,1,jumpball,4.0,[],lost,0,Ana Tadic,,REGULAR_1_09:58:00
10,09:26,09:26:00,0,2,-2,1,1,REGULAR,0,1,jumpball,NaN,[],heldball,0,NaN,,REGULAR_1_09:26:00
107,03:26,03:26:00,11,16,-5,2,1,REGULAR,19,1,substitution,NaN,[],out,0,Cassidie Burdick,,REGULAR_1_03:26:00
108,03:26,03:26:00,11,16,-5,2,1,REGULAR,5,1,substitution,NaN,[],in,0,Marieme Badiane,,REGULAR_1_03:26:00
152,00:00,00:00:00,20,23,-3,0,1,REGULAR,0,1,period,NaN,['confirmed'],end,0,NaN,ignored,REGULAR_1_00:00:00


## Récupérer les 5D et des players pour créer des colonnes binaires

In [34]:

url_match_fiba = games.iloc[22]['fiba_game_id']
game_test = fetch_game_json(url_match_fiba)
s5,pl = compile_starting_5(game_test,1)


In [35]:
equipe = 'Lyon'
games_team = fetch_team_games(games,errors,equipe)
games_team

/home/arnaud/code/Arnaud-Tiago/3.Projets/221031_LFB/lfb.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_games['starting_5'] = ''
/home/arnaud/code/Arnaud-Tiago/3.Projets/221031_LFB/lfb.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_games['players'] = ''
/home/arnaud/code/Arnaud-Tiago/3.Projets/221031_LFB/lfb.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,date,home,away,lfb_game_id,fiba_game_id,starting_5,players,all_players
3,2021-10-02,Lyon,Landerneau,/match/666473-lyon-landerneau,https://livestats.dcd.shared.geniussports.com/...,"[Aleksandra Crvendakic, Alexia Chartereau, Hel...","[Juste Jocyte, Folade Nayo Raincock Ekunwe, In...","[Kayla Alexander, Dominique Malonga, Alexia Ch..."
10,2021-10-09,Charleville-Mézières,Lyon,/match/666479-charleville-mezieres-lyon,https://livestats.dcd.shared.geniussports.com/...,"[Aleksandra Crvendakic, Alexia Chartereau, Hel...","[Juste Jocyte, Folade Nayo Raincock Ekunwe, In...","[Kayla Alexander, Dominique Malonga, Alexia Ch..."
17,2021-10-17,Lyon,Villeneuve d'Ascq,/match/666485-lyon-villeneuve-d-ascq,https://livestats.dcd.shared.geniussports.com/...,"[Aleksandra Crvendakic, Alexia Chartereau, Jul...","[Juste Jocyte, Folade Nayo Raincock Ekunwe, In...","[Kayla Alexander, Dominique Malonga, Alexia Ch..."
22,2021-10-23,Basket Landes,Lyon,/match/666491-basket-landes-lyon,https://livestats.dcd.shared.geniussports.com/...,"[Aleksandra Crvendakic, Alexia Chartereau, Hel...","[Juste Jocyte, Ingrid Tanqueray, Aleksandra Cr...","[Kayla Alexander, Dominique Malonga, Alexia Ch..."
28,2021-10-30,Lyon,Angers,/match/666497-lyon-angers,https://livestats.dcd.shared.geniussports.com/...,"[Aleksandra Crvendakic, Alexia Chartereau, Hel...","[Juste Jocyte, Ingrid Tanqueray, Aleksandra Cr...","[Kayla Alexander, Dominique Malonga, Alexia Ch..."
33,2021-11-06,Roche Vendée,Lyon,/match/666503-roche-vendee-lyon,https://livestats.dcd.shared.geniussports.com/...,"[Aleksandra Crvendakic, Alexia Chartereau, Hel...","[Juste Jocyte, Ingrid Tanqueray, Aleksandra Cr...","[Kayla Alexander, Dominique Malonga, Alexia Ch..."
36,2021-11-20,Lyon,Tarbes,/match/666506-lyon-tarbes,https://livestats.dcd.shared.geniussports.com/...,"[Ingrid Tanqueray, Aleksandra Crvendakic, Alex...","[Ingrid Tanqueray, Aleksandra Crvendakic, Alex...","[Kayla Alexander, Dominique Malonga, Alexia Ch..."
41,2021-11-27,Charnay,Lyon,/match/666511-charnay-lyon,https://livestats.dcd.shared.geniussports.com/...,"[Ingrid Tanqueray, Aleksandra Crvendakic, Alex...","[Ingrid Tanqueray, Aleksandra Crvendakic, Alex...","[Kayla Alexander, Dominique Malonga, Alexia Ch..."
48,2021-12-11,Lattes Montpellier,Lyon,/match/666519-lattes-montpellier-lyon,https://livestats.dcd.shared.geniussports.com/...,"[Aleksandra Crvendakic, Alexia Chartereau, Jul...","[Juste Jocyte, Ingrid Tanqueray, Aleksandra Cr...","[Kayla Alexander, Dominique Malonga, Alexia Ch..."
57,2021-12-18,Lyon,Bourges,/match/666525-lyon-bourges,https://livestats.dcd.shared.geniussports.com/...,"[Aleksandra Crvendakic, Alexia Chartereau, Jul...","[Juste Jocyte, Ingrid Tanqueray, Aleksandra Cr...","[Kayla Alexander, Dominique Malonga, Alexia Ch..."


## Tentative de scraping un peu vener

In [38]:
find_batch_index()

7

In [46]:
for i in range (find_batch_index()-1):
    gss_path = data_path+'games_batch_'+str(i+1)+'.csv'
    gss = pd.read_csv(gss_path).drop(columns = ['Unnamed: 0'])
    if i == 0:
        all_games = gss.copy()
    else :
        all_games = pd.merge(all_games,gss, how = 'outer')
print(all_games['id'].nunique())
all_games

13755


,id,url,home,away
0,1701860,https://livestats.dcd.shared.geniussports.com/...,Razzle,Ansett
1,1701861,https://livestats.dcd.shared.geniussports.com/...,Razzle,Ansett
2,1701862,https://livestats.dcd.shared.geniussports.com/...,TV Jets,Ansett
3,1701863,https://livestats.dcd.shared.geniussports.com/...,TV Jets,Ansett
4,1701865,https://livestats.dcd.shared.geniussports.com/...,Lightning,Ansett
...,...,...,...,...
13750,1759994,https://livestats.dcd.shared.geniussports.com/...,BC Kalev/Cramo,Tartu Ülikool Maks & Moorits
13751,1759995,https://livestats.dcd.shared.geniussports.com/...,Tartu Ülikool II,Rae Koss/Hansaviimistlus
13752,1759996,https://livestats.dcd.shared.geniussports.com/...,Eesti Maaülikool/Skarcon,G4S Noorteliiga
13753,1759997,https://livestats.dcd.shared.geniussports.com/...,Keila Korvpallikool,Tartu Kalev/Estiko
